In [78]:
import cv2
import random
import numpy as np
from ultralytics import YOLO

In [79]:
yolo = YOLO("yolov8s.pt")


In [97]:
def set_grid_white(x, y, image, cell_hw):
    image[(y * cell_hw):((y + 1 )* cell_hw), (x * cell_hw):((x + 1 )* cell_hw)] = [255, 255, 255]


# Now trying to do mean over each dim
def set_grid_mean(x, y, image, cell_hw, dims = 3):
    # get block boundaries
    row_start = y * cell_hw
    row_end   = (y + 1) * cell_hw
    col_start = x * cell_hw
    col_end   = (x + 1) * cell_hw

    # determine what to modify based on parity
    row_slice = slice(row_start, row_start + cell_hw//2) if y % 2 == 0 else slice(row_start + cell_hw//2, row_end)
    col_slice = slice(col_start, col_start + cell_hw//2) if x % 2 == 0 else slice(col_start + cell_hw//2, col_end)

    for i in range(0, dims):
        image[row_start:row_end, col_start:col_end, i] = np.mean(image[row_start:row_end, col_start:col_end, i])


# Now trying to do max over each dim
# Max behaves unexpectedly
def set_grid_max(x, y, image, cell_hw, dims = 3):
    # get block boundaries
    row_start = y * cell_hw
    row_end   = (y + 1) * cell_hw
    col_start = x * cell_hw
    col_end   = (x + 1) * cell_hw

    # determine what to modify based on parity
    row_slice = slice(row_start, row_start + cell_hw//2) if y % 2 == 0 else slice(row_start + cell_hw//2, row_end)
    col_slice = slice(col_start, col_start + cell_hw//2) if x % 2 == 0 else slice(col_start + cell_hw//2, col_end)

    for i in range(0, dims):
        image[row_start:row_end, col_start:col_end, i] = np.max(image[row_start:row_end, col_start:col_end, i])


# Now trying to do mean over each dim
def set_grid_mean_cross_mask(x, y, image, cell_hw, dims = 3):
    # get block boundaries
    row_start = y * cell_hw
    row_end   = (y + 1) * cell_hw
    col_start = x * cell_hw
    col_end   = (x + 1) * cell_hw

    # change whole grid to mean
    for i in range(0, dims):
        image[row_start:row_end, col_start:col_end, i] = np.mean(image[row_start:row_end, col_start:col_end, i])

    # determine what to part of cell to modify based on parity
    row_slice = slice(row_start, row_start + cell_hw//2) if y % 2 == 0 else slice(row_start + cell_hw//2, row_end)
    col_slice = slice(col_start, col_start + cell_hw//2) if x % 2 == 0 else slice(col_start + cell_hw//2, col_end)

    # modify said part to white
    image[row_slice, col_slice] = [255,255,255]

In [ ]:

def set_grid_mean_triangle_mask(x, y, image, cell_hw, dims = 3):
    # get block boundaries
    row_start = y * cell_hw
    row_end   = (y + 1) * cell_hw
    col_start = x * cell_hw
    col_end   = (x + 1) * cell_hw

    # change whole grid to mean
    for i in range(0, dims):
        image[row_start:row_end, col_start:col_end, i] = np.mean(image[row_start:row_end, col_start:col_end, i])

    
    if x % 2 == 0:
        if y % 2 == 0:
            for i in range(0, cell_hw):
                image[(y * cell_hw):((y + 1 )* cell_hw - i), col_start + i ] = [255, 255, 255]
        else:
            for i in range(0, cell_hw):
                image[row_start + i :((y + 1 )* cell_hw), col_start + i ] = [255, 255, 255]
    else:
        if y % 2 == 0:
            for i in range(0, cell_hw):
                image[(y * cell_hw):row_start + i, col_start + i ] = [255, 255, 255]
        else:
            for i in range(0, cell_hw):
                image[row_start + i :((y + 1 )* cell_hw), (col_end - i - 1) ] = [255, 255, 255]

In [134]:

def set_grid_mean_rand_triangle_mask(x, y, image, cell_hw, dims = 3):
    # get block boundaries
    row_start = y * cell_hw
    row_end   = (y + 1) * cell_hw
    col_start = x * cell_hw
    col_end   = (x + 1) * cell_hw

    # change whole grid to mean
    for i in range(0, dims):
        image[row_start:row_end, col_start:col_end, i] = np.mean(image[row_start:row_end, col_start:col_end, i])

    rand = random.choice([0, 1, 2, 3])
    
    if rand == 0:
        for i in range(0, cell_hw):
            image[(y * cell_hw):((y + 1 )* cell_hw - i), col_start + i ] = [255, 255, 255]
    elif rand == 1:
        for i in range(0, cell_hw):
            image[row_start + i :((y + 1 )* cell_hw), col_start + i ] = [255, 255, 255]
    elif rand == 2:
        for i in range(0, cell_hw):
            image[(y * cell_hw):row_start + i, col_start + i ] = [255, 255, 255]
    else:
        for i in range(0, cell_hw):
            image[row_start + i :((y + 1 )* cell_hw), (col_end - i - 1) ] = [255, 255, 255]

In [127]:
def get_common_divisors(x, y):
    result = []

    # determining which one is the bigger num
    smaller = x
    bigger = y

    if x > y:
        smaller = y
        bigger = x

    # start at 1 because dividing through 0 is undefined
    for i in range (1, smaller + 1):
        # so if i is a common divisor
        if (bigger % i == 0) & (smaller % i == 0):
            result.append(i)
    return result

In [136]:

# Open the default camera
cam = cv2.VideoCapture(0)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

# stores all possible cell sizes,
# current cell is stored as index i
cell_size = get_common_divisors(frame_width, frame_height)

#starts with biggest possible
i = len(cell_size) - 1
while True:
    ret, frame = cam.read()


    for x in range(0, frame_width // cell_size[i]) :
        for y in range(0, frame_height // cell_size[i]):
            set_grid_mean_rand_triangle_mask(x, y, frame, cell_size[i])

    # Write the frame to the output file
    #out.write(frame)

    # Display the captured frame
    cv2.imshow('Camera', frame)
    # Press 'q' to exit the loop
    pressed_key = cv2.waitKey(1)
    if pressed_key == ord('q'):
        break
    elif pressed_key == ord('m'):
        # making sure it doesn't go out of bound
        if i < (len(cell_size) - 1):
            i += 1
    elif pressed_key == ord('n'):
        if i > 1:
            i -= 1

# Release the capture and writer objects
cam.release()
#out.release()
cv2.destroyAllWindows()

In [125]:
temp = np.ones((2, 2)).reshape(2 , 2)
temp[:, :] = np.mean(temp[:, :])
temp[0, 0] = 255
temp    

array([[        255,           1],
       [          1,           1]])

In [ ]:
# Now trying to do mean over each dim
def set_grid_mean_triangle(x, y, image, cell_hw, dims = 3):
    for i in range(0, dims):
        image[(y * cell_hw):((y + 1 )* cell_hw), (x * cell_hw):((x + 1 )* cell_hw), i] = np.mean(image[(y * cell_hw):((y + 1 )* cell_hw), (x * cell_hw):((x + 1 )* cell_hw), i])

    
    if x % 2 == 0:
        if y % 2 == 0:
            for i in range(0, cell_hw):
                temp[(y * cell_hw):((y + 1 )* cell_hw - 1 - i), i] = [255, 255, 255]
        else:
            temp[((y * cell_hw) + (cell_hw // 2)):((y + 1 )* cell_hw), (x * cell_hw):((x + 1 )* cell_hw // 2)] = [255, 255, 255]
    else:
        if y % 2 == 0:
            temp[(y * cell_hw):((y + 1 )* cell_hw // 2), ((x * cell_hw) + (cell_hw // 2)):((x + 1 )* cell_hw)] = [255, 255, 255]
        else:
            temp[((y * cell_hw) + (cell_hw // 2)):((y + 1 )* cell_hw), ((x * cell_hw) + (cell_hw // 2)):((x + 1 )* cell_hw)] = [255, 255, 255]

In [ ]:

# x is even and y is even
temp = np.ones((4, 4)).reshape(4 , 4)
temp[0:2, 0:2] = np.mean(temp[0:2, 0:2])
temp[0, 0] = 255

# # x is even and y is odd
# temp[2:4, 0:2] = np.mean(temp[2:4, 0:2])
# temp[3, 0] = 255

# x is odd and y is odd
temp[2:4, 2:4] = np.mean(temp[2:4, 2:4])
temp[3, 3] = 255

# x is odd and y is even
temp[0:2, 2:4] = np.mean(temp[0:2, 2:4])
temp[0, 3] = 255

temp    

array([[        255,           1,           1,         255],
       [          1,           1,           1,           1],
       [          1,           1,           1,           1],
       [          1,           1,           1,           1]])

In [77]:
temp = np.ones((4, 4, 3)).reshape(4 , 4, 3)
x = 0
y = 1
cell_hw = 2

if x % 2 == 0:
    if y % 2 == 0:
        temp[(y * cell_hw):((y + 1 )* cell_hw // 2), (x * cell_hw):((x + 1 )* cell_hw // 2)] = [255, 255, 255]
    else:
        temp[((y * cell_hw) + (cell_hw // 2)):((y + 1 )* cell_hw), (x * cell_hw):((x + 1 )* cell_hw // 2)] = [255, 255, 255]
else:
    if y % 2 == 0:
        temp[(y * cell_hw):((y + 1 )* cell_hw // 2), ((x * cell_hw) + (cell_hw // 2)):((x + 1 )* cell_hw)] = [255, 255, 255]
    else:
        temp[((y * cell_hw) + (cell_hw // 2)):((y + 1 )* cell_hw), ((x * cell_hw) + (cell_hw // 2)):((x + 1 )* cell_hw)] = [255, 255, 255]
temp

array([[[          1,           1,           1],
        [          1,           1,           1],
        [          1,           1,           1],
        [          1,           1,           1]],

       [[          1,           1,           1],
        [          1,           1,           1],
        [          1,           1,           1],
        [          1,           1,           1]],

       [[          1,           1,           1],
        [          1,           1,           1],
        [          1,           1,           1],
        [          1,           1,           1]],

       [[        255,         255,         255],
        [          1,           1,           1],
        [          1,           1,           1],
        [          1,           1,           1]]])